In [40]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from valhalla import Actor, get_config


def get_isochrone(query: dict):
    result = {}
    isochrones = actor.isochrone(query)
    for contour_ix, isochrone in enumerate(isochrones["features"]):
        geom = isochrone["geometry"]["coordinates"]
        time = isochrone["properties"]["contour"]
        result[time] = Polygon(geom)
    return result


def get_walk_isochrones(location: dict[str, float], times: list[int] = [5, 10, 15, 20]):
    query = {
        "locations": [location],
        "costing": "pedestrian",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_bike_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "bicycle",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_auto_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "auto",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)

In [ ]:
CITY = "budapest"
# CSV filename in ../data/stops/{CITY}/

CITY = "madrid"
# CSV filename in ../data/stops/{CITY}/

CITY = "rotterdam"
# CSV filename in ../data/stops/{CITY}/

CITY = "paris"
# CSV filename in ../data/stops/{CITY}/

In [74]:
Path(f"../output/{CITY}").mkdir(parents=True, exist_ok=True)

In [75]:
config = get_config(
    tile_extract=f"../data/valhalla/{CITY}/valhalla_tiles.tar",
    verbose=True,
)

config["service_limits"]["isochrone"]["max_contours"] = 10
config["service_limits"]["isochrone"]["max_locations"] = 10_000
config["service_limits"]["isochrone"]["max_distance"] = 100_000

# instantiate Actor to load graph and call actions
actor = Actor(config)

2025/03/04 16:51:04.452583 [INFO] Tile extract successfully loaded with tile count: 47
2025/03/04 16:51:04.452644 [WARN] (stat): /data/valhalla/traffic.tar No such file or directory
2025/03/04 16:51:04.452647 [WARN] Traffic tile extract could not be loaded


In [ ]:
stops = pd.read_csv(f"../data/stops/{CITY}/stops_with_centrality.csv", engine="pyarrow")
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
# stops.to_crs(CRS, inplace=True)
stops.head(3)

,Node,Eigenvector Centrality,Degree Centrality,Closeness Centrality,Betweenness Centrality,stop_id,clust,stop_lat,stop_lon,geometry
0,1.0,0.000139,0.000171,0.036953,0.000000,9375373,1,49.010502,2.559527,POINT (2.55953 49.0105)
1,1.0,0.000139,0.000171,0.036953,0.000000,9375374,1,49.010502,2.559527,POINT (2.55953 49.0105)
2,86.0,0.000725,0.000856,0.038477,0.000865,3672258,86,49.003367,2.564359,POINT (2.56436 49.00337)


In [77]:
locations = []
for row in stops.itertuples():
    locations.append({"lon": row.stop_lon, "lat": row.stop_lat})

In [78]:
isochones_wide = []
isochones = []
for row in stops.itertuples():
    w = get_walk_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    b = get_bike_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    isochones.append([row.stop_id, w[5], "walk", 5])
    isochones.append([row.stop_id, w[10], "walk", 10])
    isochones.append([row.stop_id, w[15], "walk", 15])
    isochones.append([row.stop_id, b[5], "bicycle", 5])
    isochones.append([row.stop_id, b[10], "bicycle", 10])
    isochones.append([row.stop_id, b[15], "bicycle", 15])
isochones_wide = pd.DataFrame.from_records(
    isochones_wide,
    columns=[
        "stop_id",
        "walk_5",
        "walk_10",
        "walk_15",
        "bike_5",
        "bike_10",
        "bike_15",
    ],
)
isochones = pd.DataFrame.from_records(
    isochones, columns=["stop_id", "geometry", "costing", "range"]
)

In [79]:
isochones.to_csv(f"../output/{CITY}/isochrones.csv", index=False)

isochrones_gdf = gpd.GeoDataFrame(isochones, geometry="geometry", crs=4326)
isochrones_gdf.to_file(f"../output/{CITY}/isochrones.geojson")

In [80]:
# isochones = pd.read_csv(f"../output/{CITY}/isochrones.csv")
# print(isochones.head(6).to_markdown(index=False))